## Identificare code smells con approcci basati su Machine Learning

In [148]:
# Import delle librerie necessarie
import pandas as pd
import mysql.connector
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, PrecisionRecallDisplay, roc_auc_score

In [149]:
# Connessione al database
connection = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Giulia-dario1",
  database="tagman5"
)

# Verifica della connessione
if connection.is_connected():
  print("Connessione al database effettuata con successo.")
else:
  print("Errore durante la connessione al database.")

Connessione al database effettuata con successo.


#### In questa sezione troviamo la query che permette di ottenere il dataset completo, in quanto i dati sono strutturati in 5 tabelle SQL.

In [150]:
# Query SQL per unire le tabelle sample, annotation, class metrics, method metrics
sql_query = '''
SELECT *
FROM class_metrics
INNER JOIN method_metrics ON class_metrics.type_name = method_metrics.type_name
INNER JOIN annotation ON class_metrics.id = annotation.sample_id
INNER JOIN sample ON sample.id = annotation.sample_id
'''

# Esecuzione della query e conversione dei risultati in un dataframe Pandas
result = pd.read_sql_query(sql_query,connection)

C:\Users\demai\AppData\Local\Temp\ipykernel_5928\1475038157.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(sql_query,connection)


#### Stampa a video del dataset completo.

In [151]:
result

,id,dit,fanin,fanout,lcom,loc,nc,nof,nom,nopf,...,is_smell,sample_id,id,designite_id,has_smell,is_class,path_to_file,project_name,sample_constraints,smells
0,3,0,0,1,-1.0,8,0,0,1,0,...,0,3,3,379,1,1,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,2,3
1,3,0,0,1,-1.0,8,0,0,1,0,...,0,3,3,379,1,1,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,2,3
2,15,0,2,0,-1.0,4,0,0,2,0,...,0,15,15,68,1,1,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,3,3
3,15,0,2,0,-1.0,4,0,0,2,0,...,0,15,15,68,1,1,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,3,3
4,19,1,1,9,0.7142857142857143,50,0,3,7,0,...,0,19,19,72,1,1,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566847,4411,0,3,0,0.4,43,0,3,5,0,...,0,4411,4411,3106,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,1
566848,4411,0,3,0,0.4,43,0,3,5,0,...,0,4411,4411,3106,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,1
566849,4411,0,3,0,0.4,43,0,3,5,0,...,0,4411,4411,3106,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,1
566850,4389,0,1,5,0.0,16,0,2,2,0,...,0,4389,4389,1382,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,1


#### Nella successiva sezione vengono mostrate alcune caratteristiche del dataset, ad esempio i valori minimi e massimi che possono assumere alcune variabili, la media dei valori, ecc. 

In [152]:
result.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,566852.0,1575.427877,555.574719,3.0,973.0,1754.0,1974.0,7303.0
id,566852.0,33468.151502,19185.763810,6.0,19164.0,33195.0,45503.0,189131.0
id,566852.0,1739.440023,944.048873,6.0,957.0,1564.0,2511.0,11063.0
iscm,566852.0,0.001836,0.042815,0.0,0.0,0.0,0.0,1.0
isim,566852.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
islp,566852.0,0.004504,0.066959,0.0,0.0,0.0,0.0,1.0
isma,566852.0,0.991015,0.094361,0.0,1.0,1.0,1.0,1.0
is_smell,566852.0,0.005562,0.074373,0.0,0.0,0.0,0.0,1.0
sample_id,566852.0,1575.427877,555.574719,3.0,973.0,1754.0,1974.0,7303.0
id,566852.0,1575.427877,555.574719,3.0,973.0,1754.0,1974.0,7303.0


#### Da come è possibile intuire, ci sono alcuni attributi che possono essere eliminati in quanto non andranno ad influire sulla corretta o errata classificazione di un code smell. Gli attributi che verranno eliminati sono attributi in formato stringa, ad esempio il nome del pacchetto, il nome del progetto e così via.  

In [153]:
result.drop(['id', 'designite_id', 'path_to_file', 'sample_constraints', 'package_name', 'project_name', 'type_name', 'method_name'], axis=1, inplace=True)


#### Gli attributi che verranno considerati sono riportati in seguito.

In [154]:
for i in range(result.columns.size):
    print(result.columns[i])

dit
fanin
fanout
lcom
loc
nc
nof
nom
nopf
nopm
wmc
cc
loc
pc
iscm
isim
islp
isma
is_smell
sample_id
has_smell
is_class
smells


In [155]:
#stampa del dataframe
result

,dit,fanin,fanout,lcom,loc,nc,nof,nom,nopf,nopm,...,pc,iscm,isim,islp,isma,is_smell,sample_id,has_smell,is_class,smells
0,0,0,1,-1.0,8,0,0,1,0,1,...,1,0,0,0,1,0,3,1,1,3
1,0,0,1,-1.0,8,0,0,1,0,1,...,1,0,0,0,1,0,3,1,1,3
2,0,2,0,-1.0,4,0,0,2,0,0,...,1,0,0,0,1,0,15,1,1,3
3,0,2,0,-1.0,4,0,0,2,0,0,...,1,0,0,0,1,0,15,1,1,3
4,1,1,9,0.7142857142857143,50,0,3,7,0,6,...,1,0,0,0,1,0,19,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566847,0,3,0,0.4,43,0,3,5,0,5,...,1,0,0,0,0,0,4411,1,0,1
566848,0,3,0,0.4,43,0,3,5,0,5,...,2,1,0,0,0,0,4411,1,0,1
566849,0,3,0,0.4,43,0,3,5,0,5,...,2,0,0,0,0,0,4411,1,0,1
566850,0,1,5,0.0,16,0,2,2,0,2,...,2,0,0,0,0,0,4389,1,0,1


#### Il modello che verrà utilizzato è di tipo RandomForest. Tale modello prende in input solo valori numerici, poiché una buona parte degli attributi è stata definia cone VARCHAR (stringa) ho trasformato tali valori in interi.  

In [156]:
# Trasformazione dei valori 'varchar' in interi/float per la classe class_metrics (0 o 1) poiché la RandomForest prende in input solo valori numerici
result['dit'] = result['dit'].astype(int)
result['fanin'] = result['fanin'].astype(int)
result['fanout'] = result['fanout'].astype(int)
result['lcom'] = result['lcom'].astype(float).astype(int)
result['nc'] = result['nc'].astype(int)
result['nof'] = result['nof'].astype(int)
result['nom'] = result['nom'].astype(int)
result['nopf'] = result['nopf'].astype(int)
result['nopm'] = result['nopm'].astype(int)
result['wmc'] = result['wmc'].astype(int)

In [157]:
# Trasformazione dei valori 'varchar' in interi/float per la classe method_metrics (0 o 1)
result['cc'] = result['cc'].astype(int)
result['loc'] = result['loc'].astype(int)

In [158]:
# Trasformazione dei valori 'bit' in interi (0 o 1)
result['iscm'] = result['iscm'].astype(int)
result['isim'] = result['isim'].astype(int)
result['islp'] = result['islp'].astype(int)
result['isma'] = result['isma'].astype(int)
result['is_smell'] = result['is_smell'].astype(int)
result['has_smell'] = result['has_smell'].astype(int)
result['is_class'] = result['is_class'].astype(int)

##### In questo caso la variabile smells, ovvero la variabile che permette di capire la tipologia di code smells è stata definita come VARCHAR. Un'altra problematica riscontrata con tale attributo è quella relativa al valore essa può assumere; la variabile può assumere valori da 1 a 3, nel caso in cui l'istanza è stata classificata come code smell. Il primo problema si è presentato nel momento in cui il campo "smells" risultasse vuoto. Per evitare di influire in maniera negativa sulla classificazione, ho rimpiazzato la stringa vuota con il valore 0. Il secondo problema era che alcuni valori, che dovevano essere interi, in realtà si presentavano come decimali, ad esempio: 1,2. In questo caso ho risolto sosotituendo il valore 1, con il valore "1".  

In [159]:
# Trasformazione dei valori 'varchar' in interi/float per la classe sample (0 o 1) poiché la randomForest accetta solo valori numerici

def convert_to_number(val):
    if pd.isna(val):
        return 0
    elif val == '':
        return 0
    elif ',' in val:
        v = val[0]
        return float(val.replace(val, v))
    else:
        return int(val)
result['smells'] = result['smells'].apply(convert_to_number)




In [160]:
#Normalizzazione e split dei dati
# Split dei dati in variabili indipendenti e dipendenti per identificare se del codice ha o meno un code smell

# X = result.drop(['has_smell', 'smells','is_smell'], axis=1)
# y = result['has_smell']

#Split dei dati in variabili indipendenti e dipendenti per identificare di che tipologia di code smell si tratta
X = result.drop(['smells','is_smell', 'has_smell'], axis=1)
y = result['smells']

# Split dei dati in training set e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#normalizzo
normalizer = MinMaxScaler()
transformer = normalizer.fit(X_train)
X_train_normalized = transformer.transform(X_train)
X_test_normalized = transformer.transform(X_test)

In [161]:
#Modello RandomForest

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_normalized, y_train)
y_pred = rf.predict(X_test_normalized)

In [162]:
print("Numero di punti etichettati erroneamente su un totale di %d punti : %d"% (X_test_normalized.shape[0], (y_test != y_pred).sum()))

Numero di punti etichettati erroneamente su un totale di 170056 punti : 20


In [163]:
precision_score(y_test, y_pred, average='micro')

0.9998823916827398

In [164]:
accuracy_score(y_test, y_pred)

0.9998823916827398

In [165]:
recall_score(y_test, y_pred, average='micro')

0.9998823916827398

In [166]:
f1_score(y_test, y_pred, average='macro')

0.9890931858405525

In [167]:
matthews_corrcoef(y_test, y_pred)

0.9938100377087653

##### Implementare kfold validation e matrice di confusione